In [ ]:
# !pip install -r "RAG_requirement.txt"

SETUP DATA BASE ทำครั้งเดียวตอนเรื่องทำ DB ใหม่

In [ ]:
import psycopg2

conn = psycopg2.connect(
    dbname="fitderdb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
) #connect to db

cur = conn.cursor()

# cur.execute("CREATE EXTENSION vector;") # สร้าง table ไว้เก็บ vector (ทำครั้งเดียว)

cur.execute("""CREATE TABLE IF NOT EXISTS documents (
            id SERIAL PRIMARY KEY,
            content TEXT,
            embedding vector(1024)
            )""") # setting database

conn.commit()
cur.close()
conn.close()

ใส่ข้อมูลลงไปใน DATABASE เพื่อให้ model เอาไปตอบ

In [ ]:
# input ข้อมูลเข้าเป็น file txt
import psycopg2
from sentence_transformers import SentenceTransformer

# --- ส่วนการอ่านไฟล์ ---
file_path = "your_documents.txt" # เปลี่ยนเป็นพาธของไฟล์ .txt ของคุณ

# รายละเอียดการเชื่อมต่อฐานข้อมูล
conn = psycopg2.connect(
    dbname="fitderdb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# โมเดล Sentence Transformer สำหรับแปลงข้อความเป็นเวกเตอร์
embed = SentenceTransformer("BAAI/bge-m3")

def add_doc(text):
    """
    แปลงข้อความเป็นเวกเตอร์ฝังตัว (embedding) และแทรกลงในตาราง 'documents'
    """
    em = embed.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, em) VALUES (%s, %s)", (text, em))
    conn.commit() # อย่าลืมเรียก conn.commit() เป็นฟังก์ชัน!

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        # อ่านแต่ละบรรทัดจากไฟล์เป็นเอกสารแยกกัน
        for line in f:
            document_content = line.strip() # .strip() ลบช่องว่างและอักขระขึ้นบรรทัดใหม่ที่อยู่หน้า/หลัง
            if document_content: # เพิ่มเฉพาะบรรทัดที่ไม่ว่างเปล่า
                add_doc(document_content)
    print(f"ประมวลผลเอกสารจาก {file_path} สำเร็จแล้ว")
except FileNotFoundError:
    print(f"ข้อผิดพลาด: ไม่พบไฟล์ '{file_path}'")
except Exception as e:
    print(f"เกิดข้อผิดพลาด: {e}")
# --- สิ้นสุดส่วนการอ่านไฟล์ ---

cur.close()
conn.close()

In [3]:
#input ข้อมูล เป็น Array
import psycopg2
from sentence_transformers import SentenceTransformer

conn = psycopg2.connect(
    dbname="fitderdb",
    user="admin",
    password="1234",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

embedder = SentenceTransformer("BAAI/bge-m3") # ตัวแปลง text to vec

def add_document(text): #ใส่ข้อมูลเข้า db
    embedding = embedder.encode(text).tolist()
    cur.execute("INSERT INTO documents (content, embedding) VALUES (%s, %s)", (text, embedding))
    conn.commit()

documents = [
    "เปิดหลักสูตรระดับปริญญาตรี 5 สาขาวิชา ได้แก่ สาขาวิชาวิทยาการคอมพิวเตอร์ สาขาวิชานวัตกรรมดิจิทัล สาขาวิชาสารสนเทศการลงทุน สาขาวิชาคอมพิวเตอร์เกมและอีสปอร์ต และสาขาวิชาเทคโนโลยีสื่อสังคม",
    "เปิดหลักสูตรระดับปริญญาโท 3 สาขาวิชา ได้แก่ สาขาวิชาการจัดการความมั่นคงไซเบอร์และเทคโนโลยี สาขาวิชาเทคโนโลยีสื่อสังคม สาขาการจัดการนวัตกรรมดิจิทัล",
    "หลักสูตรระดับปริญญาโท สาขาการจัดการนวัตกรรมดิจิทัล เรียนแบบออนไลน์",
    "หลักสูตรระดับปริญญาโท สาขาวิชาการจัดการความมั่นคงไซเบอร์และเทคโนโลยี และสาขาวิชาเทคโนโลยีสื่อสังคม เรียนเสาร์อาทิตย์",
    "เปิดหลักสูตรระดับปริญญาเอก 2 สาขาวิชา ดังนี้ สาขาวิชาเทคโนโลยีสารสนเทศ และสาขาวิชาเทคโนโลยีสื่อสังคม",
    "คณบดีคือ รศ.ดร. เชฏฐเนติ ศรีสอ้าน",
    "วิทยาลัยเทคโนโลยีสารสนเทศและการสื่อสารแห่งมหาวิทยาลัยรังสิต ก่อตั้งเมื่อปี พ.ศ. 2537 ได้เปิดดำเนินการสอนทั้งในระดับปริญญาตรี ปริญญาโท และปริญญาเอกทางด้านสาขาวิชาที่เกี่ยวข้องทาง ด้านเทคโนโลยีสารสนเทศสำหรับกลุ่มนักศึกษา และคนทำงานที่มีพื้นฐานความรู้ที่หลากหลายและประสบการณ์ที่แตกต่างกัน เพื่อที่จะสามารถตอบสนองต่อความต้องการของสังคม",
    "ปรัชญาคือเหนือกว่าด้วยเทคโนโลยี ก้าวสู่ระดับสากล ปณิธานคือมุ่งมั่นสร้างบัณฑิตไอทีมืออาชีพสู่สังคม และวิสัยทัศน์คือเป็นผู้นำด้านให้บริการการศึกษาทางด้านเทคโนโลยีสารสนเทศ โดยเน้นการวิจัยและสร้างพันธมิตรทางวิชาการเพื่อการพัฒนาอย่างยั่งยืน"
]

for doc in documents:
    add_document(doc)

cur.close()
conn.close()

In [ ]:
#หน้าตาตัวอย่างข้อมูลที่จะนำเข้า
# documents = [
#     "เปิดหลักสูตรระดับปริญญาตรี 5 สาขาวิชา ได้แก่ สาขาวิชาวิทยาการคอมพิวเตอร์ สาขาวิชานวัตกรรมดิจิทัล สาขาวิชาสารสนเทศการลงทุน สาขาวิชาคอมพิวเตอร์เกมและอีสปอร์ต และสาขาวิชาเทคโนโลยีสื่อสังคม",
#     "เปิดหลักสูตรระดับปริญญาโท 3 สาขาวิชา ได้แก่ สาขาวิชาการจัดการความมั่นคงไซเบอร์และเทคโนโลยี สาขาวิชาเทคโนโลยีสื่อสังคม สาขาการจัดการนวัตกรรมดิจิทัล",
#     "หลักสูตรระดับปริญญาโท สาขาการจัดการนวัตกรรมดิจิทัล เรียนแบบออนไลน์",
#     "หลักสูตรระดับปริญญาโท สาขาวิชาการจัดการความมั่นคงไซเบอร์และเทคโนโลยี และสาขาวิชาเทคโนโลยีสื่อสังคม เรียนเสาร์อาทิตย์",
#     "เปิดหลักสูตรระดับปริญญาเอก 2 สาขาวิชา ดังนี้ สาขาวิชาเทคโนโลยีสารสนเทศ และสาขาวิชาเทคโนโลยีสื่อสังคม",
#     "คณบดีคือ รศ.ดร. เชฏฐเนติ ศรีสอ้าน",
#     "วิทยาลัยเทคโนโลยีสารสนเทศและการสื่อสารแห่งมหาวิทยาลัยรังสิต ก่อตั้งเมื่อปี พ.ศ. 2537 ได้เปิดดำเนินการสอนทั้งในระดับปริญญาตรี ปริญญาโท และปริญญาเอกทางด้านสาขาวิชาที่เกี่ยวข้องทาง ด้านเทคโนโลยีสารสนเทศสำหรับกลุ่มนักศึกษา และคนทำงานที่มีพื้นฐานความรู้ที่หลากหลายและประสบการณ์ที่แตกต่างกัน เพื่อที่จะสามารถตอบสนองต่อความต้องการของสังคม",
#     "ปรัชญาคือเหนือกว่าด้วยเทคโนโลยี ก้าวสู่ระดับสากล ปณิธานคือมุ่งมั่นสร้างบัณฑิตไอทีมืออาชีพสู่สังคม และวิสัยทัศน์คือเป็นผู้นำด้านให้บริการการศึกษาทางด้านเทคโนโลยีสารสนเทศ โดยเน้นการวิจัยและสร้างพันธมิตรทางวิชาการเพื่อการพัฒนาอย่างยั่งยืน"
# ]